In [1]:
import numpy as np
import math
from numpy import linalg as LA

In [2]:
def max_eigenvalue(matrix):
  w, v = LA.eig(matrix)
  return max(w)
def min_eigenvalue(matrix):
  w, v = LA.eig(matrix)
  return min(w)

assert-3 == min_eigenvalue([[1, 0, 0], [0, 2, 0], [0, 0, -3]])
assert 2 == max_eigenvalue([[1, 0, 0], [0, 2, 0], [0, 0, -3]])

In [3]:
# Вользмем матрицу из пункта k)
n = 10

A = np.zeros((n, n))
f = np.zeros((n, 1))

# a_ii = 1
# a_ij = 1 / (i + j)
# f_i = 1 / i
for i in range (0, n):
    f[i] = 1 / (i + 1)
    for j in range (0, n):
        if i == j:
            A[i][j] = 1
        else:
            A[i][j] = 1 / ((i + 1) + (j + 1))
print (np.transpose(f))
print (A)
print ("\nМаксимальное и минимальные собственные числа: %.3f, %.3f\nЧисло обуcловленности: %.3f" 
       % 
       (
           max_eigenvalue (A),
           min_eigenvalue (A),
           LA.cond (A)
       ))

[[1.         0.5        0.33333333 0.25       0.2        0.16666667
  0.14285714 0.125      0.11111111 0.1       ]]
[[1.         0.33333333 0.25       0.2        0.16666667 0.14285714
  0.125      0.11111111 0.1        0.09090909]
 [0.33333333 1.         0.2        0.16666667 0.14285714 0.125
  0.11111111 0.1        0.09090909 0.08333333]
 [0.25       0.2        1.         0.14285714 0.125      0.11111111
  0.1        0.09090909 0.08333333 0.07692308]
 [0.2        0.16666667 0.14285714 1.         0.11111111 0.1
  0.09090909 0.08333333 0.07692308 0.07142857]
 [0.16666667 0.14285714 0.125      0.11111111 1.         0.09090909
  0.08333333 0.07692308 0.07142857 0.06666667]
 [0.14285714 0.125      0.11111111 0.1        0.09090909 1.
  0.07692308 0.07142857 0.06666667 0.0625    ]
 [0.125      0.11111111 0.1        0.09090909 0.08333333 0.07692308
  1.         0.06666667 0.0625     0.05882353]
 [0.11111111 0.1        0.09090909 0.08333333 0.07692308 0.07142857
  0.06666667 1.         0.05882

In [4]:
def no_zero_minors(matrix):
    if(len(matrix) == 1 and matrix[0][0] != 0):
        return True
    else:
        little_matrix = matrix[:(len(matrix[0]) - 1), :(len(matrix[0]) - 1)]
        return (not math.isclose(LA.det(matrix), 0.0)) and no_zero_minors(little_matrix) 

def positive(matrix):
    if(len(matrix) == 1 and matrix[0][0] > 0.0):
        return True
    else:
        little_matrix = matrix[:(len(matrix[0]) - 1), :(len(matrix[0]) - 1)]
        det = LA.det(matrix)
        return LA.det(matrix) > 0.0 and positive(little_matrix) 

assert no_zero_minors(A)
assert positive(A) and np.allclose(A, np.transpose(A))
print("Все миноры не равны нулю => можно использовать LU разложение")
print("Матрица симметрична и положительно определена => применим метод Зейдаля")

Все миноры не равны нулю => можно использовать LU разложение
Матрица симметрична и положительно определена => применим метод Зейдаля


In [5]:
def get_LU(matrix):
  
    a = np.array(matrix)
    n = len(a)
 
    L  = np.zeros([n,n])
    U = np.zeros([n,n])
 
    for k in range(n-1):
                 # Выбрать k-й столбец
        gauss_vector = a[:,k]
                 # Найти значение после строки k + 1
        gauss_vector[k+1:] = gauss_vector[k+1:]/gauss_vector[k]
        gauss_vector[0:k+1] = np.zeros(k+1)
        L[:,k] = gauss_vector
        L[k,k] = 1.0
                 # Найти значение i-й строки, по которой находится матрица U
        for i in range(k+1,n):
            matrix[i,:] = matrix[i,:] - gauss_vector[i]*matrix[k,:]
        a = np.array(matrix)
    L[n-1,n-1] = 1.0
    U  = a
    return L, U
l, u = get_LU(A.copy())
assert np.allclose(A, np.dot(l, u))
#print("L:\n", l, '\n', '*'*65, "\nU:\n", u)

In [6]:
# Ly = f
# Ux = y

# triange is
# ***.....**
# 0**.....**
# 00*.....**
# 000.....0*
def solve_triangle(triangle, f):
    n = len(f)
    x = np.zeros((len(f), 1))
    x[n-1][0] = f[n-1][0]
    for i in range(n-1, -1, -1):
        x[i][0] = f[i][0] / triangle[i][i]
        
        for j in range(i):
            f[j][0] = f[j][0] - f[i][0] * triangle[j][i] / triangle[i][i]
            triangle[j][i] = triangle[j][i] - triangle[i][i] * triangle[j][i] / triangle[i][i]
        f[i][0] = 0
    return x

def solve_low_triangle(triangle, f):
    n = len(f)
    x = np.zeros((len(f), 1))
    x[n-1-1][0] = f[n-1-1][0]
    for i in range(n-1, -1, -1):
        x[n-1-i][0] = f[n-1-i][0] / triangle[n-1-i][n-1-i]
        
        for j in range(i):
            f[n-1-j][0] = f[n-1-j][0] - f[n-1-i][0] * triangle[n-1-j][n-1-i] / triangle[n-1-i][n-1-i]
            triangle[n-1-j][n-1-i] = triangle[n-1-j][n-1-i] - triangle[n-1-i][n-1-i] * triangle[n-1-j][n-1-i] / triangle[n-1-i][n-1-i]
        f[n-1-i][0] = 0
    return x


In [7]:
#Test of solvings
M_test = [[1, 2, 3, 4], [0, 2, -4, 5], [0, 0, -3, 6], [0, 0, 0, 2]]
f_test = [[1], [2], [3], [4]]
right = LA.solve(M_test.copy(), f_test.copy())
my_solve = solve_triangle(M_test.copy(), f_test.copy())
assert np.allclose(right, my_solve)

M_test = np.transpose([[1, 2, 3, 4], [0, 2, -4, 5], [0, 0, -3, 6], [0, 0, 0, 2]])
f_test = [[5], [6], [7], [8]]
right    = LA.solve      (M_test.copy(), f_test.copy())
my_solve = solve_low_triangle(M_test.copy(), f_test.copy())
assert np.allclose(right, my_solve)

In [8]:
# final solution by LU
l, u = get_LU(A.copy())
my_solution = solve_triangle(u.copy(), solve_low_triangle(l.copy(), f.copy()))
print("My solution = \n", my_solution)
assert np.allclose(my_solution, LA.solve(A, f))
print("Невязка:\n", f - np.matmul(A, my_solution))

My solution = 
 [[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]
Невязка:
 [[ 0.00000000e+00]
 [-1.11022302e-16]
 [-5.55111512e-17]
 [ 0.00000000e+00]
 [-2.77555756e-17]
 [-5.55111512e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.77555756e-17]]


In [9]:
# Верхняя релаксация
omega = 1.5
def get_L_D_U(matrix):
    n = len(matrix)
    L = np.zeros((n, n))
    D = L.copy()
    U = L.copy()
    for y in range(n):
        for x in range(n):
            if(x == y):
                D[y][x] = matrix[y][x]
            elif(x < y):
                L[y][x] = matrix[y][x]
            else:
                U[y][x] = matrix[y][x]
    return L, D, U

def get_relax(matrix, f):
    L, D, U = get_L_D_U(matrix)
    x = np.zeros((len(matrix), 1))
    y = x
    i = 0
    while LA.norm(f - matrix @ x) > 1e-2 and LA.norm(f - matrix @ y) > 1e-12:
        i += 1
        # Верхняя релаксация
        x = -LA.inv(D + omega * L) @ (((omega - 1) * D + omega * L) @ x) + omega* LA.inv(D + omega * L) @ f
        # Зейдель
        y = -LA.inv(D + L) @ U @ y + LA.inv(D + L) @ f
        if (i > 1e5):
            print("Методы не сходятся")
            print("норма разности по релаксации = ", LA.norm(f - matrix @ x))
            print("норма разности по Зейделю = ", LA.norm(f - matrix @ y))
            break
    return y, i

y, i = get_relax(A, f)
print("Количество итераций: ", i, "\nРешение:\n", y)
print("Невязка:\n", f - A @ y)

Количество итераций:  18 
Решение:
 [[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]
Невязка:
 [[-1.31228362e-13]
 [-2.15383267e-13]
 [-1.72140080e-13]
 [-1.22069022e-13]
 [-8.17401702e-14]
 [-5.19584376e-14]
 [-3.07809334e-14]
 [-1.60982339e-14]
 [-6.30051566e-15]
 [-1.38777878e-17]]
